## Migrations Processing

In [119]:
import pandas as pd
import json
from collections import defaultdict
pd.set_option('display.max_columns', None)

MIGRATION_THRESHOLD = 0 #500000

countries = {}
migrations = {'1990': {}, '1995': {}, '2000': {}, '2005': {}, '2010': {}, '2015': {}, '2017': {}}

citiesDF = pd.read_csv("data/cities.csv")
migrationsDF = pd.read_csv("data/unprocessedMigrations.csv")

# renaming columns
migrationsDF.rename(columns={'Russian Federation': 'Russia'}, inplace=True)

migrationsDF.head(20)


,Unnamed: 0,Year,Sort order,"Major area, region, country or area of destination",Notes,Code,Type of data (a),Total,Other North,Other South,Afghanistan,Albania,Algeria,American Samoa,Andorra,Angola,Anguilla,Antigua and Barbuda,Argentina,Armenia,Aruba,Australia,Austria,Azerbaijan,Bahamas,Bahrain,Bangladesh,Barbados,Belarus,Belgium,Belize,Benin,Bermuda,Bhutan,Bolivia (Plurinational State of),Bosnia and Herzegovina,Botswana,Brazil,British Virgin Islands,Brunei Darussalam,Bulgaria,Burkina Faso,Burundi,Cabo Verde,Cambodia,Cameroon,Canada,Caribbean Netherlands,Cayman Islands,Central African Republic,Chad,Channel Islands,Chile,China,"China, Hong Kong SAR","China, Macao SAR",Colombia,Comoros,Congo,Cook Islands,Costa Rica,Côte d'Ivoire,Croatia,Cuba,Curaçao,Cyprus,Czechia,Dem. People's Republic of Korea,Democratic Republic of the Congo,Denmark,Djibouti,Dominica,Dominican Republic,Ecuador,Egypt,El Salvador,Equatorial Guinea,Eritrea,Estonia,Ethiopia,Faeroe Islands,Falkland Islands (Malvinas),Fiji,Finland,France,French Guiana,French Polynesia,Gabon,Gambia,Georgia,Germany,Ghana,Gibraltar,Greece,Greenland,Grenada,Guadeloupe,Guam,Guatemala,Guinea,Guinea-Bissau,Guyana,Haiti,Holy See,Honduras,Hungary,Iceland,India,Indonesia,Iran (Islamic Republic of),Iraq,Ireland,Isle of Man,Israel,Italy,Jamaica,Japan,Jordan,Kazakhstan,Kenya,Kiribati,Kuwait,Kyrgyzstan,Lao People's Democratic Republic,Latvia,Lebanon,Lesotho,Liberia,Libya,Liechtenstein,Lithuania,Luxembourg,Madagascar,Malawi,Malaysia,Maldives,Mali,Malta,Marshall Islands,Martinique,Mauritania,Mauritius,Mayotte,Mexico,Micronesia (Fed. States of),Monaco,Mongolia,Montenegro,Montserrat,Morocco,Mozambique,Myanmar,Namibia,Nauru,Nepal,Netherlands,New Caledonia,New Zealand,Nicaragua,Niger,Nigeria,Niue,Northern Mariana Islands,Norway,Oman,Pakistan,Palau,Panama,Papua New Guinea,Paraguay,Peru,Philippines,Poland,Portugal,Puerto Rico,Qatar,Republic of Korea,Republic of Moldova,Réunion,Romania,Russia,Rwanda,Saint Helena,Saint Kitts and Nevis,Saint Lucia,Saint Pierre and Miquelon,Saint Vincent and the Grenadines,Samoa,San Marino,Sao Tome and Principe,Saudi Arabia,Senegal,Serbia,Seychelles,Sierra Leone,Singapore,Sint Maarten (Dutch part),Slovakia,Slovenia,Solomon Islands,Somalia,South Africa,South Sudan,Spain,Sri Lanka,State of Palestine,Sudan,Suriname,Swaziland,Sweden,Switzerland,Syrian Arab Republic,Tajikistan,TFYR Macedonia,Thailand,Timor-Leste,Togo,Tokelau,Tonga,Trinidad and Tobago,Tunisia,Turkey,Turkmenistan,Turks and Caicos Islands,Tuvalu,Uganda,Ukraine,United Arab Emirates,United Kingdom,United Republic of Tanzania,United States of America,United States Virgin Islands,Uruguay,Uzbekistan,Vanuatu,Venezuela (Bolivarian Republic of),Viet Nam,Wallis and Futuna Islands,Western Sahara,Yemen,Zambia,Zimbabwe
0,0,1990,1990001,WORLD,NaN,900,NaN,152542373,2144536,6342531,6724681,179490,906030,2041,3792,809942,2047,21747,430322,899683,10597,303406,505818,1634120,25172,12767,5451546,84917,1769029,365414,36114,233642,71703,28366,224687,863399,26037,500149,3094,26323,617155,1018868,337199,91681,354608,115311,997144,4325,373,46240,336916,18727,489955,4229860,551080,95648,1009148,40076,91302,17488,69724,370866,426201,835546,41878,174364,277514,38574,401962,202194,5281,42366,465022,213731,1321128,1241877,34972,170617,113928,1687517,7524,260,90156,250690,1197097,2848,3149,15208,36149,920441,3277677,373303,11994,1041455,9510,43250,5828,1376,343623,352590,55320,233660,527307,32,156553,386934,17635,6718862,1636326,629834,1506679,917639,10735,278956,3416421,588820,609032,313737,2972433,242119,4053,81482,522615,482094,215165,506600,191993,516719,76071,3430,341083,27828,58633,121365,562617,2192,647744,110735,1428,11041,134129,108385,1835,4394684,7714,4407,24155,78678,7188,1588218,2218009,685288,16057,1412,748046,728810,4151,388089,442037,132726,448460,5860,2525,138537,12534,3341574,2958,134703,3015,297925,313595,2029190,1533306,1880727,1200835,12204,1624797,625810,3087,813087,12664537,550719,884,20714,22006,485,37043,74861,1415,13941

In [120]:
# migrationsDF[migrationsDF['Major area, region, country or area of destination'] == 'Russian Federation']
migrationsDF[migrationsDF['Major area, region, country or area of destination'] == 'Russian Federation'] = migrationsDF[migrationsDF['Major area, region, country or area of destination'] == 'Russian Federation'].replace('Russian Federation', 'Russia')


In [121]:
# migrationsDF[migrationsDF['Major area, region, country or area of destination'] == 'Russian Federation'] = 'Russia'
migrationsDF[migrationsDF['Major area, region, country or area of destination'] == 'Russia']

,Unnamed: 0,Year,Sort order,"Major area, region, country or area of destination",Notes,Code,Type of data (a),Total,Other North,Other South,Afghanistan,Albania,Algeria,American Samoa,Andorra,Angola,Anguilla,Antigua and Barbuda,Argentina,Armenia,Aruba,Australia,Austria,Azerbaijan,Bahamas,Bahrain,Bangladesh,Barbados,Belarus,Belgium,Belize,Benin,Bermuda,Bhutan,Bolivia (Plurinational State of),Bosnia and Herzegovina,Botswana,Brazil,British Virgin Islands,Brunei Darussalam,Bulgaria,Burkina Faso,Burundi,Cabo Verde,Cambodia,Cameroon,Canada,Caribbean Netherlands,Cayman Islands,Central African Republic,Chad,Channel Islands,Chile,China,"China, Hong Kong SAR","China, Macao SAR",Colombia,Comoros,Congo,Cook Islands,Costa Rica,Côte d'Ivoire,Croatia,Cuba,Curaçao,Cyprus,Czechia,Dem. People's Republic of Korea,Democratic Republic of the Congo,Denmark,Djibouti,Dominica,Dominican Republic,Ecuador,Egypt,El Salvador,Equatorial Guinea,Eritrea,Estonia,Ethiopia,Faeroe Islands,Falkland Islands (Malvinas),Fiji,Finland,France,French Guiana,French Polynesia,Gabon,Gambia,Georgia,Germany,Ghana,Gibraltar,Greece,Greenland,Grenada,Guadeloupe,Guam,Guatemala,Guinea,Guinea-Bissau,Guyana,Haiti,Holy See,Honduras,Hungary,Iceland,India,Indonesia,Iran (Islamic Republic of),Iraq,Ireland,Isle of Man,Israel,Italy,Jamaica,Japan,Jordan,Kazakhstan,Kenya,Kiribati,Kuwait,Kyrgyzstan,Lao People's Democratic Republic,Latvia,Lebanon,Lesotho,Liberia,Libya,Liechtenstein,Lithuania,Luxembourg,Madagascar,Malawi,Malaysia,Maldives,Mali,Malta,Marshall Islands,Martinique,Mauritania,Mauritius,Mayotte,Mexico,Micronesia (Fed. States of),Monaco,Mongolia,Montenegro,Montserrat,Morocco,Mozambique,Myanmar,Namibia,Nauru,Nepal,Netherlands,New Caledonia,New Zealand,Nicaragua,Niger,Nigeria,Niue,Northern Mariana Islands,Norway,Oman,Pakistan,Palau,Panama,Papua New Guinea,Paraguay,Peru,Philippines,Poland,Portugal,Puerto Rico,Qatar,Republic of Korea,Republic of Moldova,Réunion,Romania,Russia,Rwanda,Saint Helena,Saint Kitts and Nevis,Saint Lucia,Saint Pierre and Miquelon,Saint Vincent and the Grenadines,Samoa,San Marino,Sao Tome and Principe,Saudi Arabia,Senegal,Serbia,Seychelles,Sierra Leone,Singapore,Sint Maarten (Dutch part),Slovakia,Slovenia,Solomon Islands,Somalia,South Africa,South Sudan,Spain,Sri Lanka,State of Palestine,Sudan,Suriname,Swaziland,Sweden,Switzerland,Syrian Arab Republic,Tajikistan,TFYR Macedonia,Thailand,Timor-Leste,Togo,Tokelau,Tonga,Trinidad and Tobago,Tunisia,Turkey,Turkmenistan,Turks and Caicos Islands,Tuvalu,Uganda,Ukraine,United Arab Emirates,United Kingdom,United Republic of Tanzania,United States of America,United States Virgin Islands,Uruguay,Uzbekistan,Vanuatu,Venezuela (Bolivarian Republic of),Viet Nam,Wallis and Futuna Islands,Western Sahara,Yemen,Zambia,Zimbabwe
140,140,1990,1990141,Russia,NaN,643,B,11524948,1428,7222,19103,1026,1303,959,..,519,304,..,582,535266,..,761,3776,936852,..,..,1280,..,809353,859,..,317,..,..,392,876,194,577,..,..,10760,525,171,52,227,738,394,..,..,131,161,..,638,67405,38,..,336,17,492,..,35,113,365,2689,..,479,13710,9699,..,2445,..,..,..,336,1499,..,29,..,62441,438,..,..,..,1737,1474,..,..,..,..,656888,144895,286,..,1997,..,..,..,..,..,429,342,..,..,..,..,18236,54,9801,377,3096,1424,194,..,2842,1020,..,655,1555,2349697,394,..,248,433625,100,95730,2133,..,..,373,..,82889,92,144,12,557,135,196,575,..,..,..,273,..,173,..,192,21142,538,..,1755,111,411,106,..,573,371,..,196,71,27,1165,..,254,1134,61,1683,..,81,..,104,784,190,31333,129,..,115,2014,292807,..,3936,..,177,..,..,..,..,..,..,31,..,317,108,..,..,..,60,..,1020,138,..,129,246,..,840,1020,1808,588,..,..,340,407,5464,424727,359,75,..,..,..,..,..,607,13032,165923,..,29,100,3301922,283,899,269,3077,..,479,890893,..,371,32251,..,111,986,354,265
410,410,1995,1995141,Russia,NaN,643,B,11928927,1466,7322,15576,988,1189,1012,..,470,300,..,572,516425,..,743,3905,905728,..,..,1035,..,886234,857,..,324,..,..,368,776,195,594,..,..,9934,568,164,50,190,624,396,..,..,132,139,..,662,64969,36,..,297,15,424,..,

In [122]:
rus1 = migrationsDF[migrationsDF['Major area, region, country or area of destination'] == 'Russia']
# rus2 = migrationsDF[migrationsDF['Major area, region, country or area of destination'] == 'Russian Federation']

rus1.shape
# rus2.shape

(7, 242)

In [123]:
for key, row in citiesDF.iterrows():
    country = row['country']
    city = row['city']
    lat = row['lat']
    lng = row['lng']

    coords = {}

    coords['country'] = country
    coords['city'] = city
    coords['latitude'] = lat
    coords['longitude'] = lng

    countries[country] = coords
# countries

In [124]:
c = 0
for key, row in migrationsDF.iterrows():
    country = row['Major area, region, country or area of destination']
    year = row['Year']
    
    if country in countries:
        for origin, migrants in row.items():
            if origin in countries and country != origin and migrants.isdigit() and int(migrants) > MIGRATION_THRESHOLD:
                migration = {}
                
                migration['origin'] = countries[origin]
                migration['destination'] = countries[country]
                migration['migrants'] = migrants
                
                ###### 
                c+=1
#                 print('Country/destination: ', country)
#                 print('Origin: ', origin)
#                 print('Migrants: ', migrants)
    
                if not country in migrations[str(year)]:
                    migrations[str(year)][country] = []
                
                migrations[str(year)][country].append(migration) 
    
#     if c > 10:
#         break
    
    
    


In [125]:
for m in migrations:
    print(len(migrations[m]))

184
184
184
184
186
186
186


In [126]:
for i in range(1995,2016,5):
#     print('\n\n\n\n\n\n==========   ', i, '   ======================================\n\n\n\n\n')
    year = migrations[str(i)]
    for country in year:
#         print('\n\n\n')
#         print("1. Current country = ", country, '\n')
#         print('2. This year flows = \n', year[country], '\n')
        for flow in year[country]:
            if country in migrations[str(i-5)]:
                prev_year_migrants = migrations[str(i-5)][country] #[flow]['migrants']
#                 print('3. Progress current flow,\n ', flow, '\n')
#                 print("4. Look if flow exists in prev year")
#                 print(prev_year_migrants)
                for c in range(0, len(prev_year_migrants)):
                    if prev_year_migrants[c]['origin']['country'] == flow['origin']['country']:
#                         print("\n 6. Progress current country = ", country)
#                         print("\n5. Country of origin = ", flow['origin']['country'])
                        actual_migrants = int(flow['migrants']) - int(prev_year_migrants[c]['migrants'])
                        
#                         print('Total migrants', i, ' = ', flow['migrants'])
#                         print('Total migrants', i-5, ' = ', prev_year_migrants[c]['migrants'])
#                         print('Actual migrants', i, ' = ', actual_migrants, '\n\n\n')
                        flow['migrants'] = actual_migrants
            else:
                print("ERROR COUNTRY NOT FOUND = ", country, ' \n YEAR = ', str(i))
            



# # migrations


5. Country of origin =  Afghanistan

 6. Progress current country =  United States of America
Total migrants 1995  =  37312
Total migrants 1990  =  28444
Actual migrants 1995  =  8868 




5. Country of origin =  Albania

 6. Progress current country =  United States of America
Total migrants 1995  =  22440
Total migrants 1990  =  5627
Actual migrants 1995  =  16813 




5. Country of origin =  Algeria

 6. Progress current country =  United States of America
Total migrants 1995  =  7859
Total migrants 1990  =  4629
Actual migrants 1995  =  3230 




5. Country of origin =  Angola

 6. Progress current country =  United States of America
Total migrants 1995  =  3352
Total migrants 1990  =  2252
Actual migrants 1995  =  1100 




5. Country of origin =  Antigua and Barbuda

 6. Progress current country =  United States of America
Total migrants 1995  =  15376
Total migrants 1990  =  12022
Actual migrants 1995  =  3354 




5. Country of origin =  Argentina

 6. Progress current country

Total migrants 1990  =  10668
Actual migrants 1995  =  3486 




5. Country of origin =  Trinidad and Tobago

 6. Progress current country =  United States of America
Total migrants 1995  =  158649
Total migrants 1990  =  115710
Actual migrants 1995  =  42939 




5. Country of origin =  Tunisia

 6. Progress current country =  United States of America
Total migrants 1995  =  5285
Total migrants 1990  =  4101
Actual migrants 1995  =  1184 




5. Country of origin =  Turkey

 6. Progress current country =  United States of America
Total migrants 1995  =  67624
Total migrants 1990  =  55087
Actual migrants 1995  =  12537 




5. Country of origin =  Turkmenistan

 6. Progress current country =  United States of America
Total migrants 1995  =  741
Total migrants 1990  =  569
Actual migrants 1995  =  172 




5. Country of origin =  Uganda

 6. Progress current country =  United States of America
Total migrants 1995  =  9334
Total migrants 1990  =  6684
Actual migrants 1995  =  2650 






5. Country of origin =  Malta

 6. Progress current country =  United States of America
Total migrants 2000  =  8777
Total migrants 1995  =  -407
Actual migrants 2000  =  9184 




5. Country of origin =  Marshall Islands

 6. Progress current country =  United States of America
Total migrants 2000  =  6478
Total migrants 1995  =  2664
Actual migrants 2000  =  3814 




5. Country of origin =  Mexico

 6. Progress current country =  United States of America
Total migrants 2000  =  9410529
Total migrants 1995  =  2529931
Actual migrants 2000  =  6880598 




5. Country of origin =  Morocco

 6. Progress current country =  United States of America
Total migrants 2000  =  35562
Total migrants 1995  =  9906
Actual migrants 2000  =  25656 




5. Country of origin =  Myanmar

 6. Progress current country =  United States of America
Total migrants 2000  =  33415
Total migrants 1995  =  6726
Actual migrants 2000  =  26689 




5. Country of origin =  Nepal

 6. Progress current country =  Uni

Actual migrants 2005  =  7529 




5. Country of origin =  Dominican Republic

 6. Progress current country =  United States of America
Total migrants 2005  =  761989
Total migrants 2000  =  528299
Actual migrants 2005  =  233690 




5. Country of origin =  Ecuador

 6. Progress current country =  United States of America
Total migrants 2005  =  364358
Total migrants 2000  =  225595
Actual migrants 2005  =  138763 




5. Country of origin =  Egypt

 6. Progress current country =  United States of America
Total migrants 2005  =  124709
Total migrants 2000  =  91532
Actual migrants 2005  =  33177 




5. Country of origin =  El Salvador

 6. Progress current country =  United States of America
Total migrants 2005  =  997858
Total migrants 2000  =  653553
Actual migrants 2005  =  344305 




5. Country of origin =  Eritrea

 6. Progress current country =  United States of America
Total migrants 2005  =  20907
Total migrants 2000  =  14578
Actual migrants 2005  =  6329 




5. Country of


5. Country of origin =  Uganda

 6. Progress current country =  United States of America
Total migrants 2005  =  22847
Total migrants 2000  =  9389
Actual migrants 2005  =  13458 




5. Country of origin =  Ukraine

 6. Progress current country =  United States of America
Total migrants 2005  =  298766
Total migrants 2000  =  218795
Actual migrants 2005  =  79971 




5. Country of origin =  United Arab Emirates

 6. Progress current country =  United States of America
Total migrants 2005  =  8252
Total migrants 2000  =  4316
Actual migrants 2005  =  3936 




5. Country of origin =  United Kingdom

 6. Progress current country =  United States of America
Total migrants 2005  =  696965
Total migrants 2000  =  667337
Actual migrants 2005  =  29628 




5. Country of origin =  Uruguay

 6. Progress current country =  United States of America
Total migrants 2005  =  37981
Total migrants 2000  =  23231
Actual migrants 2005  =  14750 




5. Country of origin =  Uzbekistan

 6. Progress c

Total migrants 2010  =  60357
Total migrants 2005  =  21431
Actual migrants 2010  =  38926 




5. Country of origin =  Myanmar

 6. Progress current country =  United States of America
Total migrants 2010  =  66988
Total migrants 2005  =  22550
Actual migrants 2010  =  44438 




5. Country of origin =  Nepal

 6. Progress current country =  United States of America
Total migrants 2010  =  48579
Total migrants 2005  =  22418
Actual migrants 2010  =  26161 




5. Country of origin =  Netherlands

 6. Progress current country =  United States of America
Total migrants 2010  =  94442
Total migrants 2005  =  -2216
Actual migrants 2010  =  96658 




5. Country of origin =  New Zealand

 6. Progress current country =  United States of America
Total migrants 2010  =  29914
Total migrants 2005  =  6775
Actual migrants 2010  =  23139 




5. Country of origin =  Nicaragua

 6. Progress current country =  United States of America
Total migrants 2010  =  254081
Total migrants 2005  =  38755
Ac

Actual migrants 2015  =  586 




5. Country of origin =  Ethiopia

 6. Progress current country =  United States of America
Total migrants 2015  =  210917
Total migrants 2010  =  96175
Actual migrants 2015  =  114742 




5. Country of origin =  Fiji

 6. Progress current country =  United States of America
Total migrants 2015  =  44586
Total migrants 2010  =  29121
Actual migrants 2015  =  15465 




5. Country of origin =  Finland

 6. Progress current country =  United States of America
Total migrants 2015  =  20835
Total migrants 2010  =  21181
Actual migrants 2015  =  -346 




5. Country of origin =  France

 6. Progress current country =  United States of America
Total migrants 2015  =  177400
Total migrants 2010  =  142266
Actual migrants 2015  =  35134 




5. Country of origin =  Georgia

 6. Progress current country =  United States of America
Total migrants 2015  =  25832
Total migrants 2010  =  14463
Actual migrants 2015  =  11369 




5. Country of origin =  Germany

 6.

In [129]:
# set minimum migrations to 0
for i in range(1995,2016,5):
    year = migrations[str(i)]
    for country in year:
        for flow in year[country]:
            if int(flow['migrants']) < 0:
                flow['migrants'] = 0
            

In [130]:
with open('data/migrations_final.json', 'w') as fp:
    json.dump(migrations, fp)